In [1]:
# May be necessary for Linux based OS
# !sudo apt-get install ffmpeg

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from process_vad_data import *

/home/fjia/anaconda3/envs/vad/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/fjia/anaconda3/envs/vad/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [8]:
import os
import sys
import glob
import json

import pickle
import wrapt
import inspect
import librosa

In [9]:
data_dir = '/home/fjia/data/freesound_resampled/'

## Process google speech command dataset

In [10]:
sc_data_folder = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/'

They have provided splited datalist. We follow that.

In [95]:
process_google_speech_train(sc_data_folder)

Overall: 105835, Train: 84849, Validatoin: 9981, Test: 11005


In [85]:
data_dir = sc_data_folder
out_dir = './manifest'

skip_num_val, seg_num_val = load_list_write_manifest(data_dir, out_dir, 'validation_list.txt', 'sc')
skip_num_test, seg_num_test = load_list_write_manifest(data_dir, out_dir, 'testing_list.txt', 'sc')
skip_num_train, seg_num_train = load_list_write_manifest(data_dir, out_dir, 'training_list.txt', 'sc')

print(f'Val: Skip {skip_num_val} samples. Get {seg_num_val} segments!')
print(f'Test: Skip {skip_num_test} samples. Get {seg_num_test} segments!')
print(f'Train: Skip {skip_num_train} samples. Get {seg_num_train} segments!')

=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 9981 to ./manifest/sc_validation_manifest.json ===
=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 11005 to ./manifest/sc_testing_manifest.json ===
=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 85245 to ./manifest/sc_training_manifest.json ===
Val: Skip 0 samples. Get 9981 segments!
Test: Skip 0 samples. Get 11005 segments!
Train: Skip 0 samples. Get 85245 segments!


In [13]:
data_dir = sc_data_folder
out_dir = './manifest'

skip_num_val, seg_num_val = load_list_write_manifest(data_dir, out_dir, 'validation_list.txt', '100ms_sc', 0.1, 0.1)
skip_num_test, seg_num_test = load_list_write_manifest(data_dir, out_dir, 'testing_list.txt', '100ms_sc', 0.1, 0.1)
skip_num_train, seg_num_train = load_list_write_manifest(data_dir, out_dir, 'training_list.txt', '100ms_sc', 0.1, 0.1)

print(f'Val: Skip {skip_num_val} samples. Get {seg_num_val} segments!')
print(f'Test: Skip {skip_num_test} samples. Get {seg_num_test} segments!')
print(f'Train: Skip {skip_num_train} samples. Get {seg_num_train} segments!')

=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 107252 to ./manifest/100ms_sc_validation_manifest.json ===
=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 118641 to ./manifest/100ms_sc_testing_manifest.json ===
=== Finished preparing manifest ! Skip 0 samples ===
=== Writing 916063 to ./manifest/100ms_sc_training_manifest.json ===
Val: Skip 0 samples. Get 107252 segments!
Test: Skip 0 samples. Get 118641 segments!
Train: Skip 0 samples. Get 916063 segments!


## Process freesound dataset

In [32]:
split_dataset_class(data_dir)

File /home/fjia/data/freesound_resampled/all.txt exists. Overwrite it!
File /home/fjia/data/freesound_resampled/background.txt exists. Overwrite it!
File /home/fjia/data/freesound_resampled/speech.txt exists. Overwrite it!
=== 5837 total samples! 1849 speech samples, 3988 background samples! ===
Finished split and write to file by class !


split_train_val_test(data_dir, 'all.txt' 0.20 ,  0.10)

In [33]:
split_train_val_test(data_dir, 'speech.txt',  0.10 ,  0.10)
split_train_val_test(data_dir, 'background.txt',  0.10 ,  0.10)

Overall: 1849, Train: 1479, Validatoin: 185, Test: 185
Finished split train, val and test for speech.txt. Write to files !
Overall: 3988, Train: 3190, Validatoin: 399, Test: 399
Finished split train, val and test for background.txt. Write to files !


duration = 60.0 would lead to skipping too many samples (2k/5k)

## freesound
### speech inside freesound / NOT USE NOW

In [34]:
data_dir = '/home/fjia/data/freesound_resampled/'
out_dir = './manifest'

skip_num_val, seg_num_val = load_list_write_manifest(data_dir, out_dir, 'speech_validation_list.txt')
skip_num_test, seg_num_test = load_list_write_manifest(data_dir, out_dir, 'speech_testing_list.txt')
skip_num_train, seg_num_train = load_list_write_manifest(data_dir, out_dir, 'speech_training_list.txt')

print(f'Val: Skip {skip_num_val} samples. Get {seg_num_val} segments!')
print(f'Test: Skip {skip_num_test} samples. Get {seg_num_test} segments!')
print(f'Train: Skip {skip_num_train} samples. Get {seg_num_train} segments!')

outdir ./manifest does not exist. Creat directory.
=== Finished preparing manifest ! Skip 43 samples ===
=== Writing 2335 to ./manifest/speech_validation_manifest.json ===
=== Finished preparing manifest ! Skip 36 samples ===
=== Writing 2643 to ./manifest/speech_testing_manifest.json ===
=== Finished preparing manifest ! Skip 282 samples ===
=== Writing 19636 to ./manifest/speech_training_manifest.json ===
Val: Skip 43 samples. Get 2335 segments!
Test: Skip 36 samples. Get 2643 segments!
Train: Skip 282 samples. Get 19636 segments!


### background inside freesound / WE WANT IT

In [20]:
data_dir = '/home/fjia/data/freesound_resampled/'
out_dir = './manifest'

skip_num_val, seg_num_val = load_list_write_manifest(data_dir, out_dir, 'background_validation_list.txt', '100ms', 0.1, 0.1)
skip_num_test, seg_num_test = load_list_write_manifest(data_dir, out_dir, 'background_testing_list.txt', '100ms', 0.1, 0.1)
skip_num_train, seg_num_train = load_list_write_manifest(data_dir, out_dir, 'background_training_list.txt', '100ms', 0.1, 0.1)

print(f'Val: Skip {skip_num_val} samples. Get {seg_num_val} segments!')
print(f'Test: Skip {skip_num_test} samples. Get {seg_num_test} segments!')
print(f'Train: Skip {skip_num_train} samples. Get {seg_num_train} segments!')

=== Finished preparing manifest ! Skip 142 samples ===
=== Writing 71190 to ./manifest/100ms_background_validation_manifest.json ===
=== Finished preparing manifest ! Skip 146 samples ===
=== Writing 67828 to ./manifest/100ms_background_testing_manifest.json ===
=== Finished preparing manifest ! Skip 1129 samples ===
=== Writing 565503 to ./manifest/100ms_background_training_manifest.json ===
Val: Skip 142 samples. Get 71190 segments!
Test: Skip 146 samples. Get 67828 segments!
Train: Skip 1129 samples. Get 565503 segments!


files = sorted(glob.glob(data_dir + '/*/*.wav'))
write_manifest(data_dir, files, manifest_name='noise_manifest_100', duration_max=1.0, filter_long=True, duration_limit=100.0)

## construct  manifest for train, val and test

## Background + Speech Command 57k, 7k, 7k

In [14]:
sc_dir = './manifest/'
get_clean_max_json(sc_dir, out_dir, '100ms_sc_training_manifest.json', 570000, '2balanced')
get_clean_max_json(sc_dir, out_dir, '100ms_sc_validation_manifest.json', 70000, '2balanced')
get_clean_max_json(sc_dir, out_dir, '100ms_sc_testing_manifest.json', 70000, '2balanced')

Get 570000 speech command to ./manifest/2balanced_100ms_sc_training_manifest.json from speech commands
Get 70000 speech command to ./manifest/2balanced_100ms_sc_validation_manifest.json from speech commands
Get 70000 speech command to ./manifest/2balanced_100ms_sc_testing_manifest.json from speech commands


In [23]:
sc_dir = './manifest/'
get_clean_max_json(sc_dir, out_dir, '100ms_background_training_manifest.json', 570000, '2balanced')
get_clean_max_json(sc_dir, out_dir, '100ms_background_validation_manifest.json', 70000, '2balanced')
get_clean_max_json(sc_dir, out_dir, '100ms_background_testing_manifest.json', 70000, '2balanced')

Get 565503/570000 speech command to ./manifest/2balanced_100ms_background_training_manifest.json from speech commands
Get 70001/70000 speech command to ./manifest/2balanced_100ms_background_validation_manifest.json from speech commands
Get 67828/70000 speech command to ./manifest/2balanced_100ms_background_testing_manifest.json from speech commands


files = sorted(glob.glob(data_dir + '/*/*.wav'))
write_manifest('.', files, manifest_name='noise_cls_manifest', duration_max=1.0, filter_long=True, duration_limit=10.0)

sc_dir = '/home/fjia/NEMO/examples/asr/notebooks/manifest'
get_clean_max_json(sc_dir, 'background_training_manifest.json', 17000)
get_clean_max_json(sc_dir, 'background_validation_manifest.json', 2000)
get_clean_max_json(sc_dir, 'background_testing_manifest.json', 5000)

## Combine test json

In [26]:
combined_des = './manifest/all_test.json'
manifest_to_combine  = './manifest/background_testing_manifest.json,./manifest/2balanced_sc_testing_manifest.json'

combine_test_set(manifest_to_combine, combined_des)

./manifest/background_testing_manifest.json
./manifest/2balanced_sc_testing_manifest.json


13886

In [5]:
data_json = os.path.join(data_dir, 'noise_manifest_100.json')
data = []
for line in open(data_json, 'r'):
    data.append(json.loads(line))    

speech = 0
background = 0

for i in data:
    if i['label'] == 'speech' :
        speech += 1
    elif i['label'] == 'background':
        background +=1
    else:
        print('something wrong!')
        
print(speech, background)

24614 71475


In [ ]:
sc_data_folder = '/home/fjia/NEMO/examples/asr/notebooks/google_dataset_v2/google_speech_recognition/v2'

sc_test_manifest.json

sc_train_manifest.json

sc_validation_manifest.json

In [25]:
def speech_command(sc_data_json):
    dataset_basedir =  '/home/fjia/NEMO/examples/asr/notebooks/google_dataset_v2/google_speech_recognition_v2/'
    data = []
    sc_seg = 0
    for line in open(os.path.join(dataset_basedir, sc_data_json), 'r'):
        data.append(json.loads(line))    
        
    print(len(data))
    return len(data)
    
train = speech_command('sc_train_manifest.json')
val = speech_command('sc_validation_manifest.json')
test = speech_command('sc_test_manifest.json')

84843
9981
11005


In [25]:
data1 = '/home/fjia/code/NeMo-fei/examples/asr/notebooks/manifest/all_test.json'

In [41]:
data = '/home/fjia/data/google_dataset_v2/google_speech_recognition_v2/test_manifest.json'